# Image Classification of Animals

**Introduction**      
Image classification is a fundamental problem in computer vision, and animal classification is a particularly challenging task due to the vast variability in species, poses, and backgrounds. To tackle this problem, we'll employ a convolutional neural network (CNN) architecture, leveraging its ability to automatically and adaptively learn spatial hierarchies of features from images. By utilizing a pre-trained CNN as a feature extractor, we'll fine-tune the model on our dataset, exploiting the transfer learning paradigm to adapt the learned representations to our specific task.

**Dataset: A Compendium of Animal Images**  
Our dataset comprises 15 classes, each corresponding to a distinct animal species, with images of size 224 x 224 x 3. This dataset presents a unique opportunity to explore the efficacy of various CNN architectures like VGG16, ResNet50, InceptionV3, MobileNetV2, EfficientNetB7 etc....

**Model Configuration**  
After conducting a thorough review of the literature and drawing from personal experience, I have selected EfficientNetB7 as the base model for transfer learning due to its impressive performance on image classification tasks, and Adam as the optimization algorithm, as I believe its adaptive learning rate and momentum capabilities will effectively navigate the complexities of our dataset and optimize the model's parameters for superior performance.

**Technical Specifications**  
-Image dimensions: 224 x 224 x 3  
-Number of classes: 15  
-CNN architecture: EfficientNetB7 (base model for transfer learning)  
-Optimization algorithm: Adam  
-Loss function: Categorical Cross-Entropy  
-Evaluation metric: Top-1 Accuracy  

**PYTHON CODE**

Importing necessary libraries


In [2]:
import random
import pandas as pd
import numpy as np
import itertools
import tensorflow as tf
from sklearn.model_selection import train_test_split

import cv2
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

from pathlib import Path
import os.path

from sklearn.metrics import classification_report, confusion_matrix


Additional Imports

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir, pred_and_plot


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


--2024-09-07 10:19:35--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: 'helper_functions.py'

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2024-09-07 10:19:35 (9.08 MB/s) - 'helper_functions.py' saved [10246/10246]



In [4]:
import tensorflow as tf
import numpy as np
import random

def seed_everything(seed=42):
    # Seed value for TensorFlow
    tf.random.set_seed(seed)

    # Seed value for NumPy
    np.random.seed(seed)

    # Seed value for Python's random library
    random.seed(seed)

    # Force TensorFlow to use single thread
    # Multiple threads are a potential source of non-reproducible results.
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threads=1,
        inter_op_parallelism_threads=1
    )

    # Make sure that TensorFlow uses a deterministic operation wherever possible
    tf.compat.v1.set_random_seed(seed)

    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)

seed_everything()


Dataset Loading and Configuration

In [5]:
# Specify the dataset directory
dataset = "/kaggle/input/animal/Animal Classification/dataset"

# Walk through the dataset directory to explore its contents
walk_through_dir(dataset)

# Set the batch size for training
batchsize = 32

# Set the target image size for resizing
targetsize = (224, 224)


There are 15 directories and 0 images in '/kaggle/input/animal/Animal Classification/dataset'.
There are 0 directories and 130 images in '/kaggle/input/animal/Animal Classification/dataset/Horse'.
There are 0 directories and 131 images in '/kaggle/input/animal/Animal Classification/dataset/Lion'.
There are 0 directories and 122 images in '/kaggle/input/animal/Animal Classification/dataset/Dog'.
There are 0 directories and 125 images in '/kaggle/input/animal/Animal Classification/dataset/Bear'.
There are 0 directories and 137 images in '/kaggle/input/animal/Animal Classification/dataset/Bird'.
There are 0 directories and 129 images in '/kaggle/input/animal/Animal Classification/dataset/Tiger'.
There are 0 directories and 126 images in '/kaggle/input/animal/Animal Classification/dataset/Kangaroo'.
There are 0 directories and 133 images in '/kaggle/input/animal/Animal Classification/dataset/Elephant'.
There are 0 directories and 137 images in '/kaggle/input/animal/Animal Classification/da

Transforming Dataset Directory into a DataFrame

In [6]:
def directory_to_dataframe(data_path):
    """
    Transforms the dataset directory into a structured table.

    Args:
        data_path (str): Path to the dataset directory.

    Returns:
        pd.DataFrame: DataFrame containing filepaths and labels.
    """
    directory = Path(data_path)

    # Gather filepaths and corresponding labels
    file_locations = list(directory.glob(r'**/*.JPG')) + list(directory.glob(r'**/*.jpg')) + list(directory.glob(r'**/*.jpeg')) + list(directory.glob(r'**/*.PNG'))

    category_labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], file_locations))

    file_series = pd.Series(file_locations, name='Filepath').astype(str)
    label_series = pd.Series(category_labels, name='Label')

    # Combine filepaths and labels into a single table
    dataset_table = pd.concat([file_series, label_series], axis=1)
    return dataset_table

# Transform the dataset directory into a DataFrame
dataset_df = directory_to_dataframe(dataset)


Handling Unidentified Image Files

In [7]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError

# Iterate over all JPEG files in the dataset directory
image_files = Path(dataset).rglob("*.jpg")

# Identify and report any files that cannot be opened as images
for file_path in image_files:
    try:
        img = PIL.Image.open(file_path)
    except PIL.UnidentifiedImageError:
        print(f"Error: Unable to open image file - {file_path}")


Splitting Data into Training and Testing Sets

In [8]:
# Divide the dataset into training and testing subsets
training_data, testing_data = train_test_split(dataset_df, test_size=0.2, shuffle=True, random_state=42)

# Define data generators for training and testing data
training_data_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)

testing_data_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)

# Create data flows for training, validation, and testing data
training_data_flow = training_data_generator.flow_from_dataframe(
    dataframe=training_data,
    x_col='Filepath',
    y_col='Label',
    target_size=targetsize,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batchsize,
    shuffle=True,
    seed=42,
    subset='training'
)

validation_data_flow = training_data_generator.flow_from_dataframe(
    dataframe=training_data,
    x_col='Filepath',
    y_col='Label',
    target_size=targetsize,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batchsize,
    shuffle=True,
    seed=42,
    subset='validation'
)

testing_data_flow = testing_data_generator.flow_from_dataframe(
    dataframe=testing_data,
    x_col='Filepath',
    y_col='Label',
    target_size=targetsize,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batchsize,
    shuffle=False
)


Found 1244 validated image filenames belonging to 15 classes.
Found 311 validated image filenames belonging to 15 classes.
Found 389 validated image filenames belonging to 15 classes.


Defining Image Augmentation Pipeline

In [9]:
# Define a sequential pipeline for image augmentation
from tensorflow.keras.layers import *
image_augmentation_pipeline = tf.keras.Sequential([
  Resizing(224, 224),  # Resize images to a fixed size
  Rescaling(1./255),  # Normalize pixel values to the range [0, 1]
  RandomFlip("horizontal"),  # Randomly flip images horizontally
  RandomRotation(0.1),  # Randomly rotate images by up to 10 degrees
  RandomZoom(0.1),  # Randomly zoom in or out by up to 10%
  RandomContrast(0.1),  # Randomly adjust image contrast by up to 10%
])


Loading Pre-Trained EfficientNetB7 Model

In [10]:
# Load the pre-trained EfficientNetB7 model
base_model = tf.keras.applications.efficientnet.EfficientNetB7(
    input_shape=(224, 224, 3),  # Input shape of the model
    include_top=False,  # Exclude the classification head
    weights='imagenet',  # Use weights pre-trained on ImageNet
    pooling='max'  # Use max pooling as the pooling layer
)

# Freeze the pre-trained model's weights
base_model.trainable = False


258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step


Defining Model Callbacks (Callbacks are functions that monitor and control the training process, allowing us to save the best model, stop training when progress stalls, and adjust the learning rate for better convergence.)

In [11]:
# Define the file path for model checkpointing
checkpoint_file_path = "animals_classification_model_checkpoint.weights.h5"

# Create a ModelCheckpoint callback to save the model's weights at each epoch
checkpoint_callback = ModelCheckpoint(
    checkpoint_file_path,
    save_weights_only=True,  # Only save the model's weights
    monitor="val_accuracy",  # Monitor the validation accuracy metric
    save_best_only=True  # Only save the best-performing model
)

# Define an EarlyStopping callback to stop training if the model's validation loss doesn't improve for 5 epochs
early_stopping_callback = EarlyStopping(
    monitor="val_loss",  # Watch the validation loss metric
    patience=5,  # Wait for 5 epochs before stopping training
    restore_best_weights=True  # Restore the best-performing model's weights
)

# Define a ReduceLROnPlateau callback to reduce the learning rate if the model's validation loss doesn't improve for 3 epochs
learning_rate_reduction_callback = ReduceLROnPlateau(
    monitor="val_loss",  # Watch the validation loss metric
    factor=0.2,  # Reduce the learning rate by a factor of 0.2
    patience=3,  # Wait for 3 epochs before reducing the learning rate
    min_lr=1e-6  # Minimum learning rate
)


Defining the Model Architecture

In [15]:
# Get the input layer of the pre-trained model
model_inputs = base_model.input

# Apply data augmentation to the input layer
augmented_inputs = image_augmentation_pipeline(model_inputs)

# Define the custom model architecture
# 1. Dense layer with 128 units, ReLU activation (output is 0 for negative inputs, f(x) = x for positive inputs, selected for its simplicity and ability to introduce non-linearity), batch normalization (normalizes inputs for each layer to have zero mean and unit variance, selected to normalize inputs and stabilize training), and dropout (selected to prevent overfitting)
x = Dense(128, activation='relu')(base_model.output)
x = BatchNormalization()(x)  # Normalizes the inputs for each layer to have zero mean and unit variance, selected to normalize inputs and stabilize training
x = Dropout(0.45)(x)  # Selected to prevent overfitting

# 2. Dense layer with 256 units, ReLU activation, and batch normalization
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)  # Normalizes the inputs for each layer to have zero mean and unit variance, selected to normalize inputs and stabilize training
x = Dropout(0.45)(x)  # Selected to prevent overfitting

# 3. Output layer with 15 units, softmax activation (outputs a probability distribution over all classes, ensuring the probabilities add up to 1, selected to output a probability distribution over all classes)
model_outputs = Dense(15, activation='softmax')(x)

# Create the custom model
custom_model = Model(inputs=model_inputs, outputs=model_outputs)

# Compile the custom model
custom_model.compile(
    optimizer=Adam(0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


Training the Model

In [18]:
# Train the model on the training data
history = custom_model.fit(
    training_data_flow,
    validation_data=validation_data_flow,
    epochs=100,  # Number of epochs to train for
    callbacks=[
        #early_stopping_callback,  # Stop training if validation loss doesn't improve
        checkpoint_callback,  # Save the best model
        learning_rate_reduction_callback  # Reduce learning rate if validation loss doesn't improve
    ]
)


Epoch 1/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 9s 206ms/step - accuracy: 0.1304 - loss: 3.5242 - val_accuracy: 0.3055 - val_loss: 2.2428 - learning_rate: 1.0000e-05
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 202ms/step - accuracy: 0.1612 - loss: 3.1465 - val_accuracy: 0.3730 - val_loss: 2.1055 - learning_rate: 1.0000e-05
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 0.1697 - loss: 3.1490 - val_accuracy: 0.4148 - val_loss: 1.9854 - learning_rate: 1.0000e-05
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 204ms/step - accuracy: 0.1838 - loss: 2.9467 - val_accuracy: 0.4469 - val_loss: 1.8669 - learning_rate: 1.0000e-05
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 204ms/step - accuracy: 0.2158 - loss: 2.8124 - val_accuracy: 0.4855 - val_loss: 1.7553 - learning_rate: 1.0000e-05
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 204ms/step - accuracy: 0.2570 - loss: 2.6995 - val_accuracy: 0.5113 - val_loss: 1.6711 - learning_rate: 1.0000e-05
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 202ms/step - acc

In [19]:
history = custom_model.fit(
    training_data_flow,
    validation_data=validation_data_flow,
    epochs=15,  # Number of epochs to train for
    callbacks=[
        #early_stopping_callback,  # Stop training if validation loss doesn't improve
        checkpoint_callback,  # Save the best model
        learning_rate_reduction_callback  # Reduce learning rate if validation loss doesn't improve
    ]
)

Epoch 1/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 205ms/step - accuracy: 0.7595 - loss: 0.8135 - val_accuracy: 0.8842 - val_loss: 0.5480 - learning_rate: 1.0000e-06
Epoch 2/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 201ms/step - accuracy: 0.7784 - loss: 0.7413 - val_accuracy: 0.8842 - val_loss: 0.5433 - learning_rate: 1.0000e-06
Epoch 3/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 202ms/step - accuracy: 0.7789 - loss: 0.7867 - val_accuracy: 0.8842 - val_loss: 0.5314 - learning_rate: 1.0000e-06
Epoch 4/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 201ms/step - accuracy: 0.7877 - loss: 0.7184 - val_accuracy: 0.8842 - val_loss: 0.5339 - learning_rate: 1.0000e-06
Epoch 5/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 202ms/step - accuracy: 0.7599 - loss: 0.7935 - val_accuracy: 0.8842 - val_loss: 0.5278 - learning_rate: 1.0000e-06
Epoch 6/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 0.8032 - loss: 0.7062 - val_accuracy: 0.8810 - val_loss: 0.5308 - learning_rate: 1.0000e-06
Epoch 7/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 

In [20]:
history = custom_model.fit(
    training_data_flow,
    validation_data=validation_data_flow,
    epochs=15,  # Number of epochs to train for
    callbacks=[
        #early_stopping_callback,  # Stop training if validation loss doesn't improve
        checkpoint_callback,  # Save the best model
        learning_rate_reduction_callback  # Reduce learning rate if validation loss doesn't improve
    ]
)

Epoch 1/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 9s 207ms/step - accuracy: 0.7896 - loss: 0.7580 - val_accuracy: 0.8842 - val_loss: 0.5192 - learning_rate: 1.0000e-06
Epoch 2/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 0.7691 - loss: 0.8204 - val_accuracy: 0.8842 - val_loss: 0.5196 - learning_rate: 1.0000e-06
Epoch 3/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 202ms/step - accuracy: 0.7769 - loss: 0.7227 - val_accuracy: 0.8875 - val_loss: 0.5148 - learning_rate: 1.0000e-06
Epoch 4/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 202ms/step - accuracy: 0.7877 - loss: 0.7784 - val_accuracy: 0.8875 - val_loss: 0.5170 - learning_rate: 1.0000e-06
Epoch 5/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 0.7755 - loss: 0.7367 - val_accuracy: 0.8875 - val_loss: 0.5124 - learning_rate: 1.0000e-06
Epoch 6/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 202ms/step - accuracy: 0.7986 - loss: 0.7600 - val_accuracy: 0.8875 - val_loss: 0.5163 - learning_rate: 1.0000e-06
Epoch 7/15
39/39 ━━━━━━━━━━━━━━━━━━━━ 8s 202ms/step - accuracy: 

Evaluating the Model

In [24]:
# Get the first batch of data and labels from the test generator
test_batch = next(iter(testing_data_flow))

# Get the true labels
true_labels = np.argmax(test_batch[1], axis=1)

# Get the predicted labels
predicted_probabilities = custom_model.predict(test_batch[0])

# Get the predicted labels
predicted_labels = np.argmax(predicted_probabilities, axis=1)

# Print the classification report
from sklearn.metrics import classification_report
print(classification_report(true_labels, predicted_labels))

# Print the confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(true_labels, predicted_labels))


1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         4
           3       1.00      1.00      1.00         2
           4       1.00      0.33      0.50         3
           5       1.00      1.00      1.00         3
           6       0.67      1.00      0.80         2
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         1
           9       0.75      1.00      0.86         3
          10       0.50      1.00      0.67         1
          11       1.00      1.00      1.00         1
          12       0.00      0.00      0.00         1
          13       1.00      1.00      1.00         3
          14       1.00      1.00      1.00         1

    accuracy                           0.91        32
   macro avg       0.86      0.89      0.8

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


The model achieved an accuracy of 0.91, indicating a strong performance on the test dataset. The macro average precision, recall, and F1-score were 0.86, 0.89, and 0.85, respectively, suggesting a good balance between precision and recall. The model performed well on most classes, with precision and recall values close to 1.0. However, class 4 had a lower recall value of 0.33, indicating some room for improvement. The weighted average precision, recall, and F1-score were also high, indicating that the model performed well across all classes. 

*For the purpose of this project, the model's performance is deemed satisfactory, and further improvements can be considered in future iterations if any.*

Saving the model file

In [26]:
# Save the custom model
custom_model.save('kaggle/working/animal_classification_model.h5')


The custom model is saved as a file named animal_classification_model.h5 in the kaggle/working directory.

In addition to saving the entire model, the best-performing model with weights as a checkpoint file during training was also saved. The main difference between these two files is: The animal_classification_model.h5 file contains the entire model architecture, including the weights and the model's configuration.The checkpoint file contains only the weights of the best-performing model, which can be used to restore the model's state and make predictions.

The animal_classification_model.h5 file can be used to: Load the entire model and make predictions, Fine-tune the model on new data, Modify the model's architecture and retrain it.

The checkpoint file can be used to: Restore the model's state and make predictions, Continue training the model from where it left off, Use the model's weights to initialize a new model